<a href="https://colab.research.google.com/github/elangbijak4/algoritma-dinda-new/blob/main/k_allay_new2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import networkx as nx

# Fungsi untuk menghitung nilai benefit antara dua negara
def benefit_function(policy_1, policy_2):
    distance = np.linalg.norm(policy_1['position'] - policy_2['position'])
    power_diff = abs(policy_1['power'] - policy_2['power'])
    benefit = -distance + 0.5 * power_diff
    return benefit, distance, power_diff

# Algoritma Optimasi Berbasis Pasangan
def pairwise_optimization(countries, K, threshold=0.5):
    N = len(countries)
    G = nx.Graph()
    G.add_nodes_from([c['id'] for c in countries])
    reasons = {}

    # Bangun graf dengan nilai benefit
    for i, country_1 in enumerate(countries):
        for j, country_2 in enumerate(countries):
            if i >= j:
                continue
            benefit, distance, power_diff = benefit_function(country_1, country_2)
            if benefit > threshold:
                G.add_edge(country_1['id'], country_2['id'])
                reasons[(country_1['id'], country_2['id'])] = {
                    'benefit': benefit,
                    'distance': distance,
                    'power_diff': power_diff
                }

    # Temukan komponen terhubung sebagai cluster awal
    initial_clusters = list(nx.connected_components(G))

    # Optimasi jika jumlah cluster tidak sesuai
    while len(initial_clusters) > K:
        distances = []
        for i, c1 in enumerate(initial_clusters):
            for j, c2 in enumerate(initial_clusters):
                if i >= j:
                    continue
                avg_benefit = np.mean([
                    benefit_function(countries[n1 - 1], countries[n2 - 1])[0]
                    for n1 in c1 for n2 in c2
                ])
                distances.append((avg_benefit, i, j))
        _, i, j = max(distances)
        initial_clusters[i].update(initial_clusters[j])
        del initial_clusters[j]

    return initial_clusters, reasons

# Data negara (contoh)
countries = [
    {'id': 1, 'position': np.array([1, 2]), 'power': 5},
    {'id': 2, 'position': np.array([2, 3]), 'power': 4},
    {'id': 3, 'position': np.array([3, 1]), 'power': 6},
    {'id': 4, 'position': np.array([6, 7]), 'power': 8},
    {'id': 5, 'position': np.array([7, 8]), 'power': 7},
]

# Jalankan Algoritma
clusters, reasons = pairwise_optimization(countries, K=2)

# Cetak Hasil
print("\nHasil Clustering Optimasi Pasangan (Diperbaiki):")
for cluster in clusters:
    print(f"\nCluster: {list(cluster)}")
    for (id1, id2), detail in reasons.items():
        if id1 in cluster and id2 in cluster:
            print(f"  - Negara {id1} dan {id2} sekutu karena:")
            print(f"    Benefit: {detail['benefit']:.2f}, Distance: {detail['distance']:.2f}, Power Difference: {detail['power_diff']:.2f}")


Hasil Clustering Optimasi Pasangan (Diperbaiki):

Cluster: [1, 2, 3]

Cluster: [4, 5]
